### RESNET ARCHITECTURE

+ Resnet is Very Deep CNN model.
+ Models are getting Deeper and Deeper, with fewer and fewer parameters.
+ The Key to train such NN is to use Skip Connections.

*WHAT IS RESIDUAL LEARNING*

+ when training the neural network, the goal is to make the model a target function $$h(x)$$
+ If yo add the input to the Output Network.
+ Then the Network will be forced to model. $$f(x) = h(x) - x$$
+ This is called Residual Learning.

*RESNET ARCHITECTURE*

+ It Starts and Ends like Google Net Architecture.
+ In between is just very deep stack of Simple Residual Units.

+ *RESIDUAL UNITS*
    
     + Composed of two Convolutional Layers.
     + No Pooling Layers.
     + with Batch Normalization.
     + Using ReLU activation function.
     + Using 3x3 Kernel.
     + Preserving Spatial Dimensions.
     
*RESNET - 34*

+ ResNet with 34 Layers.
+ Only contains the Conv Layer and Fully Connected Layer.
+ 3 RU s and 64 feature maps.
+ 4 RU s and 128 feature maps.
+ 6 RU s with 256 maps.
+ 3 RU s with 512 maps.

In [4]:
import tensorflow as tf
from tensorflow import keras
from functools import partial

In [5]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [8]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[224, 224, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 112, 112, 64)      9408      
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 residual_unit (ResidualUnit  (None, 56, 56, 64)       74240     
 )                                                               
                                                      

***